In [1]:
# Importando os módulos que serão utilizados

import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import time
import dill
import pandas as pd
import plotly.express as ply_exp
import plotly.graph_objects as ply_go

# Configurando o acesso do tensorflow aos processadores do computador
# no caso, está sendo selecionada a primeira placa de vídeo listada

# gpus = tf.config.experimental.list_physical_devices('GPU') # Listando as placas de vídeo
# tf.config.experimental.set_memory_growth(gpus[0], True)    # Selecionando a primeira GPU e configurando

# Importando os módulos contendo as funções criadas no projeto

from aux_func_V3 import *
import API_Numpy
import API_TensorFlow

In [2]:
# BI : Função de mapeamento do Henrick nas extremidades
# BI2: Função identidade nas extremidades

# nomes = [
#     "WENO-JS", "WENO-JS (M)", "WENO-JS (MS)", "WENO-JS (BI)",
#     "WENO-Z" , "WENO-Z (M)" , "WENO-Z (MS)" , "WENO-Z (BI)" , 
#     "WENO-Z+", "WENO-Z+ (M)", "WENO-Z+ (MS)", "WENO-Z+ (BI)",
#     "WENO-ZC (M)", "WENO-ZC (MS)", "WENO-ZC (BI)"
# ]
# cores = [
#     "273EAF", "5B7BD7", "8EA9F1", "BEC7FF", # WENO-JS: Azul
#     "A41127", "DD5C5B", "FF8891", "FFC2D0", # WENO-Z : Vermelho
#     "A45705", "D26F1C", "F2944A", "FFC29A", # WENO-Z+: Laranja
#     "41631D", "5B9940", "9CD87B", "DBF7C1"  # WENO-ZC: Verde
# ]

# nomes = [
#     "WENO-JS", "WENO-JS (M)", "WENO-JS (MS)", "WENO-JS (BI2)",
#     "WENO-Z" , "WENO-Z (M)" , "WENO-Z (MS)" , "WENO-Z (BI2)" , 
#     "WENO-Z+", "WENO-Z+ (M)", "WENO-Z+ (MS)", "WENO-Z+ (BI2)",
#     "WENO-ZC (M)", "WENO-ZC (MS)", "WENO-ZC (BI2)"
# ]
# cores = [
#     "#273EAF", "#5B7BD7", "#8EA9F1", "#BEC7FF", # WENO-JS: Azul
#     "#A41127", "#DD5C5B", "#FF8891", "#FFC2D0", # WENO-Z : Vermelho
#     "#A45705", "#D26F1C", "#F2944A", "#FFC29A", # WENO-Z+: Laranja
#     "#41631D", "#5B9940", "#9CD87B", "#DBF7C1"  # WENO-ZC: Verde
# ]

# API      = API_Numpy
# equation = euler_equation
# γ        = 1.4

# WENOs = {}

# WENOs[nomes[ 0]] = WENO_JS(   API, equation, γ=γ).Sim
# WENOs[nomes[ 1]] = WENO_JS_M( API, equation, γ=γ).Sim
# WENOs[nomes[ 2]] = WENO_JS_MS(API, equation, γ=γ).Sim
# WENOs[nomes[ 3]] = WENO_JS_BI(API, equation, γ=γ).Sim

# WENOs[nomes[ 4]] = WENO_Z(   API, equation, γ=γ).Sim
# WENOs[nomes[ 5]] = WENO_Z_M( API, equation, γ=γ).Sim
# WENOs[nomes[ 6]] = WENO_Z_MS(API, equation, γ=γ).Sim
# WENOs[nomes[ 7]] = WENO_Z_BI(API, equation, γ=γ).Sim

# WENOs[nomes[ 8]] = WENO_Zp(   API, equation, γ=γ).Sim
# WENOs[nomes[ 9]] = WENO_Zp_M( API, equation, γ=γ).Sim
# WENOs[nomes[10]] = WENO_Zp_MS(API, equation, γ=γ).Sim
# WENOs[nomes[11]] = WENO_Zp_BI(API, equation, γ=γ).Sim

# WENOs[nomes[12]] = WENO_ZC_M( API, equation, γ=γ).Sim
# WENOs[nomes[13]] = WENO_ZC_MS(API, equation, γ=γ).Sim
# WENOs[nomes[14]] = WENO_ZC_BI(API, equation, γ=γ).Sim

# nomes = [
#     "WENO-Z" , "WENO-Z (BI2)" ,
#     "WENO-D" , "WENO-D (BI2)" ,
#     "WENO-Z+", "WENO-ZC (BI2)"
# ]
# cores = [
#     "#273EAF", "#8EA9F1", # WENO-Z : Azul
#     "#A41127", "#FF8891", # WENO-D : Vermelho
#     "#41631D", "#9CD87B"  # WENO-ZC: Verde
# ]

# API      = API_Numpy
# equation = euler_equation
# γ        = 1.4

# WENOs = {}

# WENOs[nomes[ 0]] = simulation(API, equation, WENO_Z_scheme , γ, mapping=null_mapping   , map_function=lambda x:x  , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 1]] = simulation(API, equation, WENO_Z_scheme , γ, mapping=pre_mapping    , map_function=BI2_mapping , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 2]] = simulation(API, equation, WENO_D_scheme , γ, mapping=null_mapping   , map_function=lambda x:x  , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 3]] = simulation(API, equation, WENO_D_scheme , γ, mapping=pre_mapping    , map_function=BI2_mapping , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 4]] = simulation(API, equation, WENO_Zp_scheme, γ, mapping=null_mapping   , map_function=lambda x:x  , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 5]] = simulation(API, equation, WENO_ZC_scheme, γ, mapping=pre_inv_mapping, map_function=BI2_mapping , network=None, p=2, ε=ε_default).Sim

# nomes = [
#     "WENO-Z" , "WENO-Z (BIm)" , "WENO-Z (BI)" , "WENO-Z (BIM)" ,
#     "WENO-Z+", "WENO-ZC (BIm)", "WENO-ZC (BI)", "WENO-ZC (BIM)"
# ]
# cores = [
#     "#A41127", "#DD5C5B", "#FF8891", "#FFC2D0", # WENO-Z : Vermelho
#     "#273EAF", "#5B7BD7", "#8EA9F1", "#BEC7FF"  # WENO-ZC: Azul
# ]

# API      = API_Numpy
# equation = euler_equation
# γ        = 1.4

# WENOs = {}

# WENOs[nomes[ 0]] = simulation(API, equation, WENO_Z_scheme, γ, mapping=null_mapping, map_function=lambda x:x  , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 1]] = simulation(API, equation, WENO_Z_scheme, γ, mapping=pre_mapping , map_function=BIm_mapping , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 2]] = simulation(API, equation, WENO_Z_scheme, γ, mapping=pre_mapping , map_function=BI_mapping  , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 3]] = simulation(API, equation, WENO_Z_scheme, γ, mapping=pre_mapping , map_function=BIM_mapping , network=None, p=2, ε=ε_default).Sim

# WENOs[nomes[ 4]] = simulation(API, equation, WENO_ZC_scheme, γ, mapping=null_mapping   , map_function=lambda x:x  , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 5]] = simulation(API, equation, WENO_ZC_scheme, γ, mapping=pre_inv_mapping, map_function=BIm_mapping , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 6]] = simulation(API, equation, WENO_ZC_scheme, γ, mapping=pre_inv_mapping, map_function=BI_mapping  , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 7]] = simulation(API, equation, WENO_ZC_scheme, γ, mapping=pre_inv_mapping, map_function=BIM_mapping , network=None, p=2, ε=ε_default).Sim

nomes = [
    "WENO-Z" , "WENO-Z (BIm)" , "WENO-Z (BI)" , "WENO-Z (ENO)" ,
    "WENO-Z+", "WENO-ZC (BIm)", "WENO-ZC (BI)", "WENO-ZC (ENO)"
]
cores = [
    "#A41127", "#DD5C5B", "#FF8891", "#FFC2D0", # WENO-Z : Vermelho
    "#273EAF", "#5B7BD7", "#8EA9F1", "#BEC7FF"  # WENO-ZC: Azul
]

API      = API_Numpy
equation = euler_equation
γ        = 1.4

WENOs = {}

WENOs[nomes[ 0]] = simulation(API, equation, WENO_Z_scheme, γ, mapping=null_mapping, map_function=lambda x:x  , network=None, p=2, ε=ε_default).Sim
WENOs[nomes[ 1]] = simulation(API, equation, WENO_Z_scheme, γ, mapping=pre_mapping , map_function=BIm_mapping , network=None, p=2, ε=ε_default).Sim
WENOs[nomes[ 2]] = simulation(API, equation, WENO_Z_scheme, γ, mapping=pre_mapping , map_function=BI_mapping  , network=None, p=2, ε=ε_default).Sim
WENOs[nomes[ 3]] = simulation(API, equation, WENO_Z_scheme, γ, mapping=pre_mapping , map_function=ENO_mapping , network=None, p=2, ε=ε_default).Sim

WENOs[nomes[ 4]] = simulation(API, equation, WENO_ZC_scheme, γ, mapping=null_mapping   , map_function=lambda x:x  , network=None, p=2, ε=ε_default).Sim
WENOs[nomes[ 5]] = simulation(API, equation, WENO_ZC_scheme, γ, mapping=pre_inv_mapping, map_function=BIm_mapping , network=None, p=2, ε=ε_default).Sim
WENOs[nomes[ 6]] = simulation(API, equation, WENO_ZC_scheme, γ, mapping=pre_inv_mapping, map_function=BI_mapping  , network=None, p=2, ε=ε_default).Sim
WENOs[nomes[ 7]] = simulation(API, equation, WENO_ZC_scheme, γ, mapping=pre_inv_mapping, map_function=ENO_mapping , network=None, p=2, ε=ε_default).Sim


In [3]:

def cond_inicial(problema):
    
    if problema == "1-Lax":
        
        # Riemann problem of Lax
        #-------------------------------------------------------------------
        t_final   = 0.13
        n         = 200
        interval  = [-0.5, 0.5]
        fator     = 40
        n_ref     = n*fator
        fronteira = FronteiraFixa
        #-------------------------------------------------------------------

    if problema == "2-Sod":

        # Riemann problem of Sod
        #-------------------------------------------------------------------
        t_final   = 2
        n         = 200
        interval  = [-5, 5]
        fator     = 40
        n_ref     = n*fator
        fronteira = FronteiraFixa
        #-------------------------------------------------------------------

    if problema == "3-Shock Density Wave Titarev-Toro":
        
        # Shock density wave Titarev–Toro
        #-------------------------------------------------------------------
        t_final   = 5
        n         = 1500
        interval  = [-10, 5]
        fator     = 8
        n_ref     = n*fator
        fronteira = TitarevToroBoundary
        #-------------------------------------------------------------------
        
    if problema == "4-Shock Density Wave Shu-Osher":
        
        # Shock density wave Shu–Osher
        #-------------------------------------------------------------------
        t_final   = 1.8
        n         = 200
        interval  = [-5, 5]
        fator     = 40
        n_ref     = n*fator
        fronteira = FronteiraFixa
        #-------------------------------------------------------------------
        
    if problema == "5-Blast Wave":
        
        # Blast wave
        #-------------------------------------------------------------------
        t_final   = 0.038
        n         = 400
        interval  = [0, 1]
        fator     = 8
        n_ref     = n*fator
        fronteira = FronteiraReflexiva
        #-------------------------------------------------------------------
        
    Δx     = (interval[1]-interval[0])/n
    Δx_ref = (interval[1]-interval[0])/n_ref

    x     = tf.range(interval[0], interval[1], Δx    , dtype=dtype)
    x_ref = tf.range(interval[0], interval[1], Δx_ref, dtype=dtype)

    CFL = 0.5
    γ   = 1.4

    if problema == "1-Lax":
        
        # Riemann problem of Lax
        #-------------------------------------------------------------------
        def f_test(x):
            ρ = np.where(x < 0, 0.445, 0.5)
            u = np.where(x < 0, 0.698, 0.0)
            p = np.where(x < 0, 3.528, 0.5710)
            E = p/(γ-1)+ρ*(u**2)/2
            Q = np.stack([ρ,ρ*u,E], axis=0)
            return Q

        Q     = f_test(x)
        Q_ref = f_test(x_ref)
        #-------------------------------------------------------------------

    if problema == "2-Sod":
        
        # Riemann problem of Sod
        #-------------------------------------------------------------------
        def f_test(x):
            ρ = np.where(x < 0, 0.125, 1)
            u = x-x
            p = np.where(x < 0, 0.1  , 1)
            E = p/(γ-1)+ρ*(u**2)/2
            Q = np.stack([ρ,ρ*u,E], axis=0)
            return Q
        
        Q     = f_test(x)
        Q_ref = f_test(x_ref)
        #-------------------------------------------------------------------

    if problema == "3-Shock Density Wave Titarev-Toro":
    
        # Shock density wave Titarev–Toro
        #-------------------------------------------------------------------
        def f_test(x):
            k = 20
            ρ = np.where(x < -4.5, 1.515695, 1+np.sin(k*np.pi*x)/10)
            u = np.where(x < -4.5, 0.523346, 0)
            p = np.where(x < -4.5, 1.805000, 1)
            E = p/(γ-1)+ρ*(u**2)/2
            Q = np.stack([ρ,ρ*u,E], axis=0)
            return Q
        
        Q     = f_test(x)
        Q_ref = f_test(x_ref)
        #-------------------------------------------------------------------

    if problema == "4-Shock Density Wave Shu-Osher":
                
        # Shock density wave Shu–Osher
        #-------------------------------------------------------------------
        def f_test(x):
            k = 5
            ρ = np.where(x < -4, 3.857143, 1+0.2*np.sin(k*x))
            u = np.where(x < -4, 2.629369, 0)
            p = np.where(x < -4, 31/3    , 1)
            E = p/(γ-1)+ρ*(u**2)/2
            Q = np.stack([ρ,ρ*u,E], axis=0)
            return Q
        
        Q     = f_test(x)
        Q_ref = f_test(x_ref)
        #-------------------------------------------------------------------

    if problema == "5-Blast Wave":
        
        # Blast wave
        #-------------------------------------------------------------------
        def f_test(x):
            ρ = x-x+1
            u = x-x
            p = np.where(  x < 0.1, 1000, 0.01)
            p = np.where(0.9 <   x,  100,    p)
            E = p/(γ-1)+ρ*(u**2)/2
            Q = np.stack([ρ,ρ*u,E], axis=0)
            return Q
        
        Q     = f_test(x)
        Q_ref = f_test(x_ref)
        #-------------------------------------------------------------------
    
    init = {
        "Q"        : Q         ,
        "Q_ref"    : Q_ref     ,
        "t_final"  : t_final   ,
        "Δx"       : Δx        ,
        "Δx_ref"   : Δx_ref    ,
        "CFL"      : CFL       ,
        "fronteira": fronteira ,
        "interval" : interval  ,
        "fator"    : fator
    }
    
    return init

In [7]:
problemas = [
    "1-Lax"                            ,
    "2-Sod"                            ,
    "3-Shock Density Wave Titarev-Toro",
    "4-Shock Density Wave Shu-Osher"   ,
    "5-Blast Wave"
]

# problemas = ["4-Shock Density Wave Shu-Osher"]

u = {}
for problema in problemas:
    
    init = cond_inicial(problema)
    
    Q         = init["Q"]
    Q_ref     = init["Q_ref"]
    t_final   = init["t_final"]
    Δx        = init["Δx"]
    Δx_ref    = init["Δx_ref"]
    CFL       = init["CFL"]
    fronteira = init["fronteira"]
    
    for nome in nomes: 
        
        try:
            u[problema + ": " + nome] = WENOs[nome](Q, t_final, Δx, CFL, fronteira)
            print(problema[2:] + ": passou o " + nome)
        except:
            u[problema + ": " + nome] = Q[...]
            print(problema[2:] + ": NÃO passou o " + nome)
    
    try:
        u[problema + ": referência"] = WENOs["WENO-Z"](Q_ref, t_final, Δx_ref, CFL, fronteira)
        print(problema[2:] + ": referência foi obtida")
    except:
        u[problema + ": referência"] = Q_ref[...]
        print(problema[2:] + ": referência NÃO foi obtida")
    

Lax: passou o WENO-Z7
Lax: passou o WENO-Z (BIm)
Lax: passou o WENO-Z (BI)
Lax: passou o WENO-Z (ENO)
Lax: passou o WENO-Z+
Lax: passou o WENO-ZC (BIm)
Lax: passou o WENO-ZC (BI)
Lax: passou o WENO-ZC (ENO)
Lax: referência foi obtida
Sod: passou o WENO-Z
Sod: passou o WENO-Z (BIm)
Sod: passou o WENO-Z (BI)
Sod: passou o WENO-Z (ENO)
Sod: passou o WENO-Z+
Sod: passou o WENO-ZC (BIm)
Sod: passou o WENO-ZC (BI)
Sod: passou o WENO-ZC (ENO)
Sod: referência foi obtida
Shock Density Wave Titarev-Toro: passou o WENO-Z
Shock Density Wave Titarev-Toro: passou o WENO-Z (BIm)
Shock Density Wave Titarev-Toro: passou o WENO-Z (BI)
Shock Density Wave Titarev-Toro: passou o WENO-Z (ENO)
Shock Density Wave Titarev-Toro: passou o WENO-Z+
Shock Density Wave Titarev-Toro: passou o WENO-ZC (BIm)
Shock Density Wave Titarev-Toro: passou o WENO-ZC (BI)
Shock Density Wave Titarev-Toro: passou o WENO-ZC (ENO)


C:\Users\JAQUELINE\Documents\GitHub\Top_mat_A\WENO-DS\aux_equation.py:150: RuntimeWarning:

invalid value encountered in sqrt



Shock Density Wave Titarev-Toro: referência foi obtida
Shock Density Wave Shu-Osher: passou o WENO-Z
Shock Density Wave Shu-Osher: passou o WENO-Z (BIm)
Shock Density Wave Shu-Osher: passou o WENO-Z (BI)
Shock Density Wave Shu-Osher: passou o WENO-Z (ENO)
Shock Density Wave Shu-Osher: passou o WENO-Z+
Shock Density Wave Shu-Osher: passou o WENO-ZC (BIm)
Shock Density Wave Shu-Osher: passou o WENO-ZC (BI)
Shock Density Wave Shu-Osher: passou o WENO-ZC (ENO)
Shock Density Wave Shu-Osher: referência foi obtida
Blast Wave: passou o WENO-Z
Blast Wave: passou o WENO-Z (BIm)
Blast Wave: passou o WENO-Z (BI)
Blast Wave: passou o WENO-Z (ENO)
Blast Wave: passou o WENO-Z+
Blast Wave: passou o WENO-ZC (BIm)
Blast Wave: passou o WENO-ZC (BI)
Blast Wave: passou o WENO-ZC (ENO)
Blast Wave: referência foi obtida


In [10]:
df    = {}
x     = {}
y     = {}
label = {}

for problema in problemas:
    
    init     = cond_inicial(problema)
    interval = init["interval"]
    Δx       = init["Δx"]
    
    x[problema]     = tf.range(interval[0], interval[1], Δx, dtype=dtype)
    y[problema]     = u[problema + ": " + nomes[0]][0]
    label[problema] = tf.repeat([nomes[0]], [len(u[problema + ": " + nomes[0]][0])])

    for i in tf.range(1, len(nomes)):
        x[problema]     = tf.concat([x[problema]    , tf.range(interval[0], interval[1], Δx, dtype=dtype)], axis = 0)
        y[problema]     = tf.concat([y[problema]    , u[problema + ": " + nomes[i]][0]], axis = 0)
        label[problema] = tf.concat([label[problema], tf.repeat([nomes[i]], [len(u[problema + ": " + nomes[i]][0])])], axis = 0)
    
    df[problema] = pd.DataFrame(
        {
            "x"     : x[problema]     ,
            "y"     : y[problema]     ,
            "label" : label[problema]
        }
    )

In [11]:
for problema in problemas:
    
    init     = cond_inicial(problema)
    interval = init["interval"]
    Δx_ref   = init["Δx_ref"]
    
    fig = ply_exp.line(
        data_frame = df[problema]    ,
        x          = x[problema]     ,
        y          = y[problema]     ,
        color      = label[problema] ,
        color_discrete_sequence = cores
    )
    fig.add_trace(
        ply_go.Scatter(
            x    = tf.range(interval[0], interval[1], Δx_ref, dtype=dtype),
            y    = u[problema + ": referência"][0]      ,
            mode = "lines"                              ,
            line = ply_go.scatter.Line(color="#666666") ,
            name = "Referência (WENO-Z)"
        )
    )
    fig.update_yaxes(exponentformat="power")
    # fig.show()
    fig.write_html("htmls/" + problema[2:] + " WENO (ENO).html")

In [9]:
Weights = {}

Weights[nomes[ 0]] = WENO_JS(   API, equation, γ=γ).Get_weights
Weights[nomes[ 1]] = WENO_JS_M( API, equation, γ=γ).Get_weights
Weights[nomes[ 2]] = WENO_JS_MS(API, equation, γ=γ).Get_weights
Weights[nomes[ 3]] = WENO_JS_BI(API, equation, γ=γ).Get_weights

Weights[nomes[ 4]] = WENO_Z(   API, equation, γ=γ).Get_weights
Weights[nomes[ 5]] = WENO_Z_M( API, equation, γ=γ).Get_weights
Weights[nomes[ 6]] = WENO_Z_MS(API, equation, γ=γ).Get_weights
Weights[nomes[ 7]] = WENO_Z_BI(API, equation, γ=γ).Get_weights

Weights[nomes[ 8]] = WENO_Zp(   API, equation, γ=γ).Get_weights
Weights[nomes[ 9]] = WENO_Zp_M( API, equation, γ=γ).Get_weights
Weights[nomes[10]] = WENO_Zp_MS(API, equation, γ=γ).Get_weights
Weights[nomes[11]] = WENO_Zp_BI(API, equation, γ=γ).Get_weights

Weights[nomes[12]] = WENO_ZC_M( API, equation, γ=γ).Get_weights
Weights[nomes[13]] = WENO_ZC_MS(API, equation, γ=γ).Get_weights
Weights[nomes[14]] = WENO_ZC_BI(API, equation, γ=γ).Get_weights

In [10]:
u_ref = {}
pesos = {}

for problema in problemas:
    
    init = cond_inicial(problema)
    
    fator     = init["fator"]
    Δx        = init["Δx"]
    fronteira = init["fronteira"]
    
    u_ref[problema] = u[problema + ": referência"]
#     u_ref[problema] = u_ref[problema][:,np.arange(0, len(u_ref[problema][0]), fator)]
    
    for nome in nomes:
        pesos[problema + ": " + nome] = Weights[nome](u_ref[problema], Δx, fronteira)[0][0]

In [11]:
for problema in problemas:
    
    init     = cond_inicial(problema)
    interval = init["interval"]
#     Δx       = init["Δx"]
#     aux_x    = tf.range(interval[0], interval[1], Δx, dtype=dtype)
    Δx_ref       = init["Δx_ref"]
    aux_x    = tf.range(interval[0], interval[1], Δx_ref, dtype=dtype)

    for nome in nomes:
        
        aux = problema + ": " + nome
        
        label_0 = tf.repeat([nome + ": omega 0"], [len(pesos[aux][:,0])])
        label_1 = tf.repeat([nome + ": omega 1"], [len(pesos[aux][:,1])])
        label_2 = tf.repeat([nome + ": omega 2"], [len(pesos[aux][:,2])])

        x     = tf.concat([          aux_x,           aux_x,           aux_x], axis = 0)
        y     = tf.concat([pesos[aux][:,0], pesos[aux][:,1], pesos[aux][:,2]], axis = 0)
        label = tf.concat([        label_0,         label_1,         label_2], axis = 0)
        
        df = pd.DataFrame(
            {
                "x"     : x     ,
                "y"     : y     ,
                "label" : label
            }
        )
        fig = ply_exp.line(
            data_frame = df    ,
            x          = x     ,
            y          = y     ,
            color      = label ,
            log_y      = True
        )
        fig.add_trace(
            ply_go.Scatter(
                x    = aux_x                                ,
                y    = np.exp(u_ref[problema][0])           ,
                mode = "lines"                              ,
                line = ply_go.scatter.Line(color="#666666") ,
                name = "Função",
            )
        )
        fig.update_yaxes(exponentformat="power")
#         fig.show()
        fig.write_html("htmls/Pesos para " + problema[2:] + " com " + nome + " WENO-D (referência).html")

ValueError: All arrays must be of the same length

In [49]:
for problema in problemas:
    
    init     = cond_inicial(problema)
    interval = init["interval"]
#     Δx       = init["Δx"]
#     aux_x    = tf.range(interval[0], interval[1], Δx, dtype=dtype)
    Δx_ref       = init["Δx_ref"]
    aux_x    = tf.range(interval[0], interval[1], Δx_ref, dtype=dtype)

    for i in np.arange(3):
        
        aux = problema + ": " + nomes[0]
        
        x     = aux_x
        y     = pesos[aux][:,i]
        label = tf.repeat(["omega " + str(i) + ": " + nomes[0]], [len(pesos[aux])])

        for j in range(1, len(nomes)):
            
            aux = problema + ": " + nomes[j]
            
            x     = tf.concat([    x, aux_x], axis = 0)
            y     = tf.concat([    y, pesos[aux][:,i]], axis = 0)
            label = tf.concat([label, tf.repeat(["omega " + str(i) + ": " + nomes[j]], [len(pesos[aux])])], axis = 0)

        df = pd.DataFrame(
            {
                "x"     : x     ,
                "y"     : y     ,
                "label" : label
            }
        )
        fig = ply_exp.line(
            data_frame = df    ,
            x          = x     ,
            y          = y     ,
            color      = label ,
            log_y      = True
        )
        fig.add_trace(
            ply_go.Scatter(
                x    = aux_x                                ,
                y    = np.exp(u_ref[problema][0])           ,
                mode = "lines"                              ,
                line = ply_go.scatter.Line(color="#666666") ,
                name = "Função"                             ,
            )
        )
        fig.update_yaxes(exponentformat="power")
#         fig.show()
        fig.write_html("htmls/Peso " + str(i) + " para " + problema[2:] + " WENO-D (referência).html")